In [1]:
import kenlm
#import spacy
import os

import pandas as pd
import numpy as np
 
PATH = "C:\Users\barka\Desktop\NLP\Data\"

In [2]:
#model = kenlm.LanguageModel(r"C:\Users\Louis\kenlm\lm\test.arpa")
model = kenlm.LanguageModel(r"C:\Users\barka\Desktop\NLP\Model\model_2.arpa")

In [6]:
sample = pd.read_table(r"C:\Users\barka\Desktop\NLP\Data\test.txt", header=None)

#f = open(r"C:\Users\barka\Desktop\NLP\Data\test.txt", "r", encoding="utf-8")

In [10]:
f = open(r"C:\Users\barka\Desktop\NLP\Data\test.txt", "r", encoding="utf-8")

sentences = f.readlines()[:1000]

In [18]:
scores = []
for sentence in sentences:
    sentence_filtered = sentence.replace('\n', '')
    score = model.score(sentence_filtered)
    scores.append(score)

In [15]:
sentence_filtered

'He did not release any figures on civilian casualties .nothing'

In [19]:
scores

[-34.35860061645508,
 -29.051578521728516,
 -55.8439826965332,
 -78.04142761230469,
 -44.529659271240234,
 -75.87973022460938,
 -51.24210739135742,
 -9.06804084777832,
 -24.101337432861328,
 -93.20549011230469,
 -101.1080093383789,
 -23.805696487426758,
 -97.17626190185547,
 -41.49275588989258,
 -76.23064422607422,
 -16.501047134399414,
 -50.01478576660156,
 -148.4837188720703,
 -34.71936798095703,
 -42.20796203613281,
 -22.464839935302734,
 -13.908744812011719,
 -27.402284622192383,
 -39.89973068237305,
 -37.93867492675781,
 -27.187517166137695,
 -82.03949737548828,
 -53.545692443847656,
 -128.23388671875,
 -60.86665725708008,
 -22.767932891845703,
 -88.12931060791016,
 -46.734867095947266,
 -75.5098876953125,
 -70.69247436523438,
 -88.26985931396484,
 -42.21452331542969,
 -77.33345031738281,
 -56.59836196899414,
 -112.35929870605469,
 -38.88132858276367,
 -82.98603057861328,
 -51.64929962158203,
 -66.53104400634766,
 -20.803131103515625,
 -69.51996612548828,
 -97.8185043334961,
 -26.